# Neural Networks Assignment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/homer_bart.zip

/content/drive/MyDrive/homer_bart.zip


In [17]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=24aeb41922bb1e137cfbf59e61378d52fd66a21fd85a19a2988a27a84f57d121
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

"""
# Unzip the file
file_name = '/content/drive/MyDrive/homer_bart.zip'
with ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/homer_bart')
"""

# Define the path and parameters
image_folder_path_bart = '/content/drive/MyDrive/homer_bart/Bart'
image_folder_path_homer = '/content/drive/MyDrive/homer_bart/Homer'
img_height, img_width = 64, 64

# Load and convert images to NumPy arrays
def load_images_from_folder(folder, img_height, img_width):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

images_bart = load_images_from_folder(image_folder_path_bart, img_height, img_width)
images_homer = load_images_from_folder(image_folder_path_homer, img_height, img_width)

# Normalize the image data
images_bart = images_bart / 255.0  # Normalize pixel values to [0, 1]
images_homer = images_homer / 255.0

# Split data into test and train sets
X_train_bart, X_test_bart = train_test_split(images_bart, test_size=0.1, random_state=42)
X_train_homer, X_test_homer = train_test_split(images_homer, test_size=0.1, random_state=42)

print("Training data shape for bart:", X_train_bart.shape)
print("Validation data shape for bart:", X_test_bart.shape)
print("Training data shape for homer:", X_train_homer.shape)
print("Validation data shape for homer:", X_test_homer.shape)


Training data shape for bart: (144, 64, 64, 3)
Validation data shape for bart: (16, 64, 64, 3)
Training data shape for homer: (98, 64, 64, 3)
Validation data shape for homer: (11, 64, 64, 3)


In [31]:
Y_train_bart = np.zeros(X_train_bart.shape[0])
Y_test_bart = np.zeros(X_test_bart.shape[0])
Y_train_homer = np.ones(X_train_homer.shape[0])
Y_test_homer = np.ones(X_test_homer.shape[0])

In [32]:
# Need to shuffle the arrays
# For now I'll concatenate

X_train = np.concatenate((X_train_bart, X_train_homer), axis=0)
Y_train = np.concatenate((Y_train_bart, Y_train_homer), axis=0)
X_test = np.concatenate((X_test_bart, X_test_homer), axis=0)
Y_test = np.concatenate((Y_test_bart, Y_test_homer), axis=0)

In [33]:
Y_train = Y_train.reshape(-1,1)
Y_train.shape
Y_test = Y_test.reshape(-1,1)
Y_test.shape

(27, 1)

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import utils

In [46]:
model = Sequential()
model.add(Flatten(input_shape=(64, 64, 3)))  # Flatten the input
model.add(Dense(24,activation='relu', input_shape=(64,64,3)))
model.add(Dense(24, activation='relu'))
model.add(Dense(18, activation='relu'))
# output layer
model.add(Dense(1, activation='sigmoid'))

# compiling the sequential model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 12288)             0         
                                                                 
 dense_38 (Dense)            (None, 24)                294936    
                                                                 
 dense_39 (Dense)            (None, 24)                600       
                                                                 
 dense_40 (Dense)            (None, 18)                450       
                                                                 
 dense_41 (Dense)            (None, 1)                 19        
                                                                 
Total params: 296005 (1.13 MB)
Trainable params: 296005 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
model.fit(X_train, Y_train, epochs=50, verbose=0, batch_size=32)

In [50]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 81.48%
